In [85]:
import pandas as pd
import math
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import RocCurveDisplay
import joblib

from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.over_sampling import SVMSMOTE

from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import NearMiss
from imblearn.under_sampling import RepeatedEditedNearestNeighbours
from imblearn.under_sampling import OneSidedSelection

from imblearn.combine import SMOTEENN
from imblearn.combine import SMOTETomek

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

In [2]:
data_1718 = pd.read_csv('train_and_test_dataset\combined_data_1718.csv', sep=',')
data_1718

,Rndrng_Prvdr_State_FIPS,Tot_HCPCS_Cds,Tot_Benes_PartB,Tot_Srvcs,Tot_Sbmtd_Chrg,Tot_Mdcr_Pymt_Amt,Bene_Avg_Risk_Scre,Exclusion,Rndrng_Prvdr_Gndr_F,Rndrng_Prvdr_Gndr_M,...,Tot_Suplr_HCPCS_Cds,Tot_Suplr_Benes,Tot_Suplr_Clms,Tot_Suplr_Srvcs,Suplr_Sbmtd_Chrgs,Suplr_Mdcr_Pymt_Amt,Rndrng_Prvdr_Type_Intensive Cardiac Rehabilitation,Rndrng_Prvdr_Type_Medical Genetics and Genomics,Rndrng_Prvdr_Type_Medicare Diabetes Preventive Program,Rndrng_Prvdr_Type_Undersea and Hyperbaric Medicine
0,37.0,53,941,6053.0,330786.64,234031.17,4.5499,0,0,1,...,18.0,28.0,234.0,38432.0,283882.15,49679.51,0.0,0.0,0.0,0.0
1,12.0,39,1780,5902.0,1557600.00,582036.98,3.1298,0,0,1,...,24.0,21.0,144.0,19204.0,57531.29,16456.49,0.0,0.0,0.0,0.0
2,31.0,143,487,13646.0,934781.00,371989.62,1.0452,0,0,1,...,65.0,66.0,449.0,13778.0,211657.34,73806.30,0.0,0.0,0.0,0.0
3,9.0,25,516,2201.0,230000.00,92868.98,1.0939,0,0,1,...,47.0,50.0,203.0,2906.0,41810.34,10083.63,0.0,0.0,0.0,0.0
4,26.0,86,2000,5962.0,963080.00,332887.04,2.6301,0,0,1,...,4.0,28.0,11.0,311.0,17205.04,6789.74,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
563738,12.0,31,160,604.0,124013.80,41210.91,0.9733,0,0,1,...,21.0,12.0,38.0,139.0,10540.64,2828.07,0.0,0.0,0.0,0.0
563739,12.0,42,1293,7652.0,853391.00,448715.39,1.2065,0,0,1,...,79.0,115.0,565.0,13089.0,142784.40,35013.80,0.0,0.0,0.0,0.0
563740,12.0,24,62,141.0,21722.00,6707.42,0.9904,0,1,0,...,7.0,28.0,16.0,335.0,1082.06,149.66,0.0,0.0,0.0,0.0
563741,6.0,50,334,1111.0,181414.40,65362.61,1.3348,0,1,0,...,28.0,16.0,72.0,608.0,13805.37,3176.26,0.0,0.0,0.0,0.0


In [24]:
data_1718['Exclusion'].value_counts()

0    563768
1        54
Name: Exclusion, dtype: int64

In [25]:
#Remove data with letters mixed in some values
new_data =  data_1718.loc[data_1718['Rfrg_Prvdr_State_FIPS'].replace('[A-Za-z]',np.nan,regex=True).dropna().index]

In [26]:
new_data['Exclusion'].value_counts()

0    563689
1        54
Name: Exclusion, dtype: int64

In [27]:
data_19['Exclusion'].value_counts()

0    285240
1        15
Name: Exclusion, dtype: int64

In [28]:
#Remove data with letters mixed in some values
new_data_2 =  data_19.loc[data_19['Rfrg_Prvdr_State_FIPS'].replace('[A-Za-z]',np.nan,regex=True).dropna().index]

In [29]:
new_data_2['Exclusion'].value_counts()

0    285209
1        15
Name: Exclusion, dtype: int64

In [32]:
new_data.to_csv('train_and_test_dataset\combined_data_1718.csv', index = False)
new_data_2.to_csv('train_and_test_dataset\combined_data_19.csv', index = False)

In [3]:
data_19 = pd.read_csv('train_and_test_dataset\combined_data_19.csv', sep=',')
column_ordered_list = [column for column in data_1718]
data_19 = data_19.loc[:, column_ordered_list]
data_19

,Rndrng_Prvdr_State_FIPS,Tot_HCPCS_Cds,Tot_Benes_PartB,Tot_Srvcs,Tot_Sbmtd_Chrg,Tot_Mdcr_Pymt_Amt,Bene_Avg_Risk_Scre,Exclusion,Rndrng_Prvdr_Gndr_F,Rndrng_Prvdr_Gndr_M,...,Tot_Suplr_HCPCS_Cds,Tot_Suplr_Benes,Tot_Suplr_Clms,Tot_Suplr_Srvcs,Suplr_Sbmtd_Chrgs,Suplr_Mdcr_Pymt_Amt,Rndrng_Prvdr_Type_Intensive Cardiac Rehabilitation,Rndrng_Prvdr_Type_Medical Genetics and Genomics,Rndrng_Prvdr_Type_Medicare Diabetes Preventive Program,Rndrng_Prvdr_Type_Undersea and Hyperbaric Medicine
0,23.0,12,81,97.0,26523.61,10310.43,2.3971,0,1,0,...,2.0,27.0,22.0,22.0,2416.94,206.82,0,0,0,0
1,12.0,10,478,2200.0,251257.08,108854.08,3.1036,0,1,0,...,29.0,12.0,35.0,1073.0,21152.53,4989.90,0,0,0,0
2,27.0,4,115,425.0,192569.00,39088.12,2.2260,0,1,0,...,14.0,27.0,31.0,88.0,7399.31,816.80,0,0,0,0
3,6.0,23,93,232.0,112254.00,20353.74,3.6583,0,0,1,...,3.0,27.0,16.0,22.0,5198.38,536.16,0,0,0,0
4,39.0,15,124,571.0,51715.00,15847.96,3.4528,0,1,0,...,32.0,30.0,96.0,4551.0,94467.84,27423.34,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285092,12.0,43,130,601.0,116555.79,43330.56,0.9795,0,0,1,...,25.0,14.0,57.0,356.0,14225.17,3270.40,0,0,0,0
285093,12.0,35,1320,7403.0,849187.00,463528.14,1.2510,0,0,1,...,79.0,108.0,632.0,15842.0,202589.15,45630.23,0,0,0,0
285094,12.0,28,108,197.0,40321.46,8318.18,1.4878,0,1,0,...,5.0,27.0,16.0,331.0,926.79,43.26,0,0,0,0
285095,6.0,44,408,1239.0,205365.50,73037.90,1.3153,0,1,0,...,30.0,18.0,53.0,572.0,14068.52,3461.25,0,0,0,0


In [4]:
#The 2019 data was used as the test set
y_test = data_19['Exclusion'].astype('int')
x_test = data_19.drop(labels = 'Exclusion',axis = 1)

In [7]:
#the first sampling method -- RandomOverSampler
y = data_1718['Exclusion'].astype('int')
x = data_1718.drop(labels = 'Exclusion',axis = 1)

In [8]:
#This is the random upsampling method
#This is the first sampling method
ros = RandomOverSampler(sampling_strategy='auto', random_state = None)
x_resampled_ros, y_resampled_ros = ros.fit_resample(x, y)
x_resampled_ros.to_csv('resampled_datasets/x_resampled_ros.csv',index=False)
y_resampled_ros.to_csv('resampled_datasets/y_resampled_ros.csv',index=False)

In [49]:
# C = data_19.dropna(axis=0, how='any')
# C

,Rndrng_Prvdr_State_FIPS,Tot_HCPCS_Cds,Tot_Benes_PartB,Tot_Srvcs,Tot_Sbmtd_Chrg,Tot_Mdcr_Pymt_Amt,Bene_Avg_Risk_Scre,Exclusion,Rndrng_Prvdr_Gndr_F,Rndrng_Prvdr_Gndr_M,...,Tot_Drug_Cst,Tot_Benes_PartD,Rfrg_Prvdr_State_FIPS,Tot_Suplrs,Tot_Suplr_HCPCS_Cds,Tot_Suplr_Benes,Tot_Suplr_Clms,Tot_Suplr_Srvcs,Suplr_Sbmtd_Chrgs,Suplr_Mdcr_Pymt_Amt
0,23.0,12,81,97.0,26523.61,10310.43,2.3971,0,1,0,...,8145.46,66.0,23,1.0,2.0,27.0,22.0,22.0,2416.94,206.82
1,12.0,10,478,2200.0,251257.08,108854.08,3.1036,0,1,0,...,31594.29,101.0,12,6.0,29.0,12.0,35.0,1073.0,21152.53,4989.90
2,27.0,4,115,425.0,192569.00,39088.12,2.2260,0,1,0,...,14911.73,99.0,27,3.0,14.0,27.0,31.0,88.0,7399.31,816.80
3,6.0,23,93,232.0,112254.00,20353.74,3.6583,0,0,1,...,9958.75,25.0,6,2.0,3.0,27.0,16.0,22.0,5198.38,536.16
4,39.0,15,124,571.0,51715.00,15847.96,3.4528,0,1,0,...,3365.41,80.0,39,11.0,32.0,30.0,96.0,4551.0,94467.84,27423.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285219,12.0,43,130,601.0,116555.79,43330.56,0.9795,0,0,1,...,75727.54,128.0,12,14.0,25.0,14.0,57.0,356.0,14225.17,3270.40
285220,12.0,35,1320,7403.0,849187.00,463528.14,1.2510,0,0,1,...,840408.21,1008.0,12,62.0,79.0,108.0,632.0,15842.0,202589.15,45630.23
285221,12.0,28,108,197.0,40321.46,8318.18,1.4878,0,1,0,...,17685.42,97.0,12,3.0,5.0,27.0,16.0,331.0,926.79,43.26
285222,6.0,44,408,1239.0,205365.50,73037.90,1.3153,0,1,0,...,333681.06,556.0,6,15.0,30.0,18.0,53.0,572.0,14068.52,3461.25


In [51]:
# C.to_csv('train_and_test_dataset\combined_data_19.csv', index = False)

In [9]:
#Random Forset Classifier under the first resampling method

rdc_ros = RandomForestClassifier(random_state = None)
rdc_ros = rdc_ros.fit(x_resampled_ros, y_resampled_ros)
joblib.dump(rdc_ros, 'trained_models/rdc_ros.pkl')

['trained_models/rdc_ros.pkl']

In [44]:
#logistic regression under the first resampling method

lr_ros = LogisticRegression(random_state=None)
lr_ros = lr_ros.fit(x_resampled_ros, y_resampled_ros)
joblib.dump(lr_ros, 'trained_models/lr_ros.pkl')

D:\anaconda3\envs\MLBDenv\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


['trained_models/lr_ros.pkl']

In [31]:
#Naive Bayes under the first resampling method

gnb_ros = GaussianNB()
gnb_ros = gnb_ros.fit(x_resampled_ros, y_resampled_ros)
joblib.dump(gnb_ros, 'trained_models/gnb_ros.pkl')

['trained_models/gnb_ros.pkl']

In [11]:
#the second resampling method, RandomUnderSampler

rus = RandomUnderSampler(random_state=None)
x_resampled_rus, y_resampled_rus = rus.fit_resample(x, y)
x_resampled_rus.to_csv('resampled_datasets/x_resampled_rus.csv',index=False)
y_resampled_rus.to_csv('resampled_datasets/y_resampled_rus.csv',index=False)

In [12]:
#Random Forset Classifier under the second resampling method

rdc_rus = RandomForestClassifier(random_state = None)
rdc_rus = rdc_rus.fit(x_resampled_rus, y_resampled_rus)
joblib.dump(rdc_rus, 'trained_models/rdc_rus.pkl')

['trained_models/rdc_rus.pkl']

In [45]:
#logistic regression under the second resampling method

lr_rus = LogisticRegression(random_state=None)
lr_rus = lr_rus.fit(x_resampled_rus, y_resampled_rus)
joblib.dump(lr_rus, 'trained_models/lr_rus.pkl')

D:\anaconda3\envs\MLBDenv\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


['trained_models/lr_rus.pkl']

In [32]:
#Naive Bayes under the second resampling method

gnb_rus = GaussianNB()
gnb_rus = gnb_rus.fit(x_resampled_rus, y_resampled_rus)
joblib.dump(gnb_rus, 'trained_models/gnb_rus.pkl')

['trained_models/gnb_rus.pkl']

In [14]:
#the third resampling method, SMOTE

smo = SMOTE(sampling_strategy='auto',random_state=None)
x_resampled_smo, y_resampled_smo = smo.fit_resample(x, y)
x_resampled_smo.to_csv('resampled_datasets/x_resampled_smo.csv',index=False)
y_resampled_smo.to_csv('resampled_datasets/y_resampled_smo.csv',index=False)

In [15]:
#Random Forset Classifier under the third resampling method

rdc_smo = RandomForestClassifier(random_state = None)
rdc_smo = rdc_smo.fit(x_resampled_smo, y_resampled_smo)
joblib.dump(rdc_smo, 'trained_models/rdc_smo.pkl')

['trained_models/rdc_smo.pkl']

In [46]:
#logistic regression under the third resampling method

lr_smo = LogisticRegression(random_state=None)
lr_smo = lr_smo.fit(x_resampled_smo, y_resampled_smo)
joblib.dump(lr_smo, 'trained_models/lr_smo.pkl')

D:\anaconda3\envs\MLBDenv\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


['trained_models/lr_smo.pkl']

In [33]:
#Naive Bayes under the third resampling method

gnb_smo = GaussianNB()
gnb_smo = gnb_smo.fit(x_resampled_smo, y_resampled_smo)
joblib.dump(gnb_smo, 'trained_models/gnb_smo.pkl')

['trained_models/gnb_smo.pkl']

In [16]:
#the fourth resampling method, BorderlineSMOTE

bdlsmo = BorderlineSMOTE(sampling_strategy='auto',random_state=None,  kind='borderline-1') #kind='borderline-1' is the default choice
x_resampled_bdlsmo, y_resampled_bdlsmo = bdlsmo.fit_resample(x, y)
x_resampled_bdlsmo.to_csv('resampled_datasets/x_resampled_bdlsmo.csv',index=False)
y_resampled_bdlsmo.to_csv('resampled_datasets/y_resampled_bdlsmo.csv',index=False)

In [17]:
#Random Forset Classifier under the fourth resampling method

rdc_bdlsmo = RandomForestClassifier(random_state = None)
rdc_bdlsmo = rdc_bdlsmo.fit(x_resampled_bdlsmo, y_resampled_bdlsmo)
joblib.dump(rdc_bdlsmo, 'trained_models/rdc_bdlsmo.pkl')

['trained_models/rdc_bdlsmo.pkl']

In [47]:
#logistic regression under the fourth resampling method

lr_bdlsmo = LogisticRegression(random_state=None)
lr_bdlsmo = lr_bdlsmo.fit(x_resampled_bdlsmo, y_resampled_bdlsmo)
joblib.dump(lr_bdlsmo, 'trained_models/lr_bdlsmo.pkl')

D:\anaconda3\envs\MLBDenv\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


['trained_models/lr_bdlsmo.pkl']

In [34]:
#Naive Bayes under the fourth resampling method

gnb_bdlsmo = GaussianNB()
gnb_bdlsmo = gnb_bdlsmo.fit(x_resampled_bdlsmo, y_resampled_bdlsmo)
joblib.dump(gnb_bdlsmo, 'trained_models/gnb_bdlsmo.pkl')

['trained_models/gnb_bdlsmo.pkl']

In [18]:
#the fifth resampling method, SVMSMOTE

svmsmo = SVMSMOTE(sampling_strategy='auto',random_state=None)
x_resampled_svmsmo, y_resampled_svmsmo = svmsmo.fit_resample(x, y)
x_resampled_svmsmo.to_csv('resampled_datasets/x_resampled_svmsmo.csv',index=False)
y_resampled_svmsmo.to_csv('resampled_datasets/y_resampled_svmsmo.csv',index=False)

In [19]:
#Random Forset Classifier under the fifth resampling method

rdc_svmsmo = RandomForestClassifier(random_state = None)
rdc_svmsmo = rdc_svmsmo.fit(x_resampled_svmsmo, y_resampled_svmsmo)
joblib.dump(rdc_svmsmo, 'trained_models/rdc_svmsmo.pkl')

['trained_models/rdc_svmsmo.pkl']

In [48]:
#logistic regression under the fifth resampling method

lr_svmsmo = LogisticRegression(random_state=None)
lr_svmsmo = lr_svmsmo.fit(x_resampled_svmsmo, y_resampled_svmsmo)
joblib.dump(lr_svmsmo, 'trained_models/lr_svmsmo.pkl')

D:\anaconda3\envs\MLBDenv\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


['trained_models/lr_svmsmo.pkl']

In [35]:
#Naive Bayes under the fifth resampling method

gnb_svmsmo = GaussianNB()
gnb_svmsmo = gnb_svmsmo.fit(x_resampled_svmsmo, y_resampled_svmsmo)
joblib.dump(gnb_svmsmo, 'trained_models/gnb_svmsmo.pkl')

['trained_models/gnb_svmsmo.pkl']

In [20]:
#the sixth resampling method, SMOTEENN

smote_enn = SMOTEENN(random_state=None)
x_resampled_smote_enn, y_resampled_smote_enn = smote_enn.fit_resample(x, y)
x_resampled_smote_enn.to_csv('resampled_datasets/x_resampled_smote_enn.csv',index=False)
y_resampled_smote_enn.to_csv('resampled_datasets/y_resampled_smote_enn.csv',index=False)

In [21]:
#Random Forset Classifier under the sixth resampling method

rdc_smote_enn = RandomForestClassifier(random_state = None)
rdc_smote_enn = rdc_smote_enn.fit(x_resampled_smote_enn, y_resampled_smote_enn)
joblib.dump(rdc_smote_enn, 'trained_models/rdc_smote_enn.pkl')

['trained_models/rdc_smote_enn.pkl']

In [49]:
#ogistic regression under the sixth resampling method

lr_smote_enn = LogisticRegression(random_state=None)
lr_smote_enn = lr_smote_enn.fit(x_resampled_smote_enn, y_resampled_smote_enn)
joblib.dump(lr_smote_enn, 'trained_models/lr_smote_enn.pkl')

D:\anaconda3\envs\MLBDenv\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


['trained_models/lr_smote_enn.pkl']

In [36]:
#Naive Bayes under the sixth resampling method

gnb_smote_enn = GaussianNB()
gnb_smote_enn = gnb_smote_enn.fit(x_resampled_smote_enn, y_resampled_smote_enn)
joblib.dump(gnb_smote_enn, 'trained_models/gnb_smote_enn.pkl')

['trained_models/gnb_smote_enn.pkl']

In [22]:
#the seventh resampling method, SMOTETomek

smote_tomek = SMOTETomek(sampling_strategy='auto',random_state=None)
x_resampled_smote_tomek, y_resampled_smote_tomek = smote_tomek.fit_resample(x, y)
x_resampled_smote_tomek.to_csv('resampled_datasets/x_resampled_smote_tomek.csv',index=False)
y_resampled_smote_tomek.to_csv('resampled_datasets/y_resampled_smote_tomek.csv',index=False)

In [23]:
#Random Forset Classifier under the seventh resampling method

rdc_smote_tomek = RandomForestClassifier(random_state = None)
rdc_smote_tomek = rdc_smote_tomek.fit(x_resampled_smote_tomek, y_resampled_smote_tomek)
joblib.dump(rdc_smote_tomek, 'trained_models/rdc_smote_tomek.pkl')

['trained_models/rdc_smote_tomek.pkl']

In [50]:
#logistic regression under the seventh resampling method

lr_smote_tomek = LogisticRegression(random_state=None)
lr_smote_tomek = lr_smote_tomek.fit(x_resampled_smote_tomek, y_resampled_smote_tomek)
joblib.dump(lr_smote_tomek, 'trained_models/lr_smote_tomek.pkl')

D:\anaconda3\envs\MLBDenv\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


['trained_models/lr_smote_tomek.pkl']

In [37]:
#Naive Bayes under the seventh resampling method

gnb_smote_tomek = GaussianNB()
gnb_smote_tomek = gnb_smote_tomek.fit(x_resampled_smote_tomek, y_resampled_smote_tomek)
joblib.dump(gnb_smote_tomek, 'trained_models/gnb_smote_tomek.pkl')

['trained_models/gnb_smote_tomek.pkl']

In [24]:
#the eighth resampling method, NearMiss

nm = NearMiss(sampling_strategy='auto', version=3)
x_resampled_nm, y_resampled_nm = nm.fit_resample(x, y)
x_resampled_nm.to_csv('resampled_datasets/x_resampled_nm.csv',index=False)
y_resampled_nm.to_csv('resampled_datasets/y_resampled_nm.csv',index=False)

In [25]:
#Random Forset Classifier under the eighth resampling method

rdc_nm = RandomForestClassifier(random_state = None)
rdc_nm = rdc_nm.fit(x_resampled_nm, y_resampled_nm)
joblib.dump(rdc_nm, 'trained_models/rdc_nm.pkl')

['trained_models/rdc_nm.pkl']

In [51]:
#logistic regression under the eighth resampling method

lr_nm = LogisticRegression(random_state=None)
lr_nm = lr_nm.fit(x_resampled_nm, y_resampled_nm)
joblib.dump(lr_nm, 'trained_models/lr_nm.pkl')

D:\anaconda3\envs\MLBDenv\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


['trained_models/lr_nm.pkl']

In [38]:
#Naive Bayes under the eighth resampling method

gnb_nm = GaussianNB()
gnb_nm = gnb_nm.fit(x_resampled_nm, y_resampled_nm)
joblib.dump(gnb_nm, 'trained_models/gnb_nm.pkl')

['trained_models/gnb_nm.pkl']

In [26]:
#the ninth resampling method, RepeatedEditedNearestNeighbours

renn = RepeatedEditedNearestNeighbours(kind_sel='all',max_iter=100)
x_resampled_renn, y_resampled_renn = renn.fit_resample(x, y)
x_resampled_renn.to_csv('resampled_datasets/x_resampled_renn.csv',index=False)
y_resampled_renn.to_csv('resampled_datasets/y_resampled_renn.csv',index=False)

In [27]:
#Random Forset Classifier under the ninth resampling method

rdc_renn = RandomForestClassifier(random_state = None)
rdc_renn = rdc_renn.fit(x_resampled_renn, y_resampled_renn)
joblib.dump(rdc_renn, 'trained_models/rdc_renn.pkl')

['trained_models/rdc_renn.pkl']

In [52]:
#logistic regression under the ninth resampling method

lr_renn = LogisticRegression(random_state=None)
lr_renn = lr_renn.fit(x_resampled_renn, y_resampled_renn)
joblib.dump(lr_renn, 'trained_models/lr_renn.pkl')

D:\anaconda3\envs\MLBDenv\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


['trained_models/lr_renn.pkl']

In [39]:
#Naive Bayes under the ninth resampling method

gnb_renn = GaussianNB()
gnb_renn = gnb_renn.fit(x_resampled_renn, y_resampled_renn)
joblib.dump(gnb_renn, 'trained_models/gnb_renn.pkl')

['trained_models/gnb_renn.pkl']

In [28]:
#the tenth resampling method, OneSidedSelection

oss = OneSidedSelection(sampling_strategy='auto',random_state=None)
x_resampled_oss, y_resampled_oss = oss.fit_resample(x, y)
x_resampled_oss.to_csv('resampled_datasets/x_resampled_oss.csv',index=False)
y_resampled_oss.to_csv('resampled_datasets/y_resampled_oss.csv',index=False)

In [29]:
#Random Forset Classifier under the tenth resampling method

rdc_oss = RandomForestClassifier(random_state = None)
rdc_oss = rdc_oss.fit(x_resampled_oss, y_resampled_oss)
joblib.dump(rdc_oss, 'trained_models/rdc_oss.pkl')

['trained_models/rdc_oss.pkl']

In [53]:
#logistic regression under the tenth resampling method

lr_oss = LogisticRegression(random_state=None)
lr_oss = lr_oss.fit(x_resampled_oss, y_resampled_oss)
joblib.dump(lr_oss, 'trained_models/lr_oss.pkl')

D:\anaconda3\envs\MLBDenv\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


['trained_models/lr_oss.pkl']

In [40]:
#Naive Bayes under the tenth resampling method

gnb_oss = GaussianNB()
gnb_oss = gnb_oss.fit(x_resampled_oss, y_resampled_oss)
joblib.dump(gnb_oss, 'trained_models/gnb_oss.pkl')

['trained_models/gnb_oss.pkl']